<a href="https://colab.research.google.com/github/tejaswini16256/30-Days-6-Companies/blob/main/final_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import zipfile

# Function to extract dataset from zip file
def extract_dataset(zip_path, extract_to="dataset"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return os.path.join(extract_to, "2DSagittal")

# Example usage
zip_path = "/content/2DSagittal.zip"
dataset_dir = extract_dataset(zip_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/2DSagittal.zip'

In [6]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.applications import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import backend as K
from imblearn.combine import SMOTETomek
from sklearn.svm import SVC

# Skull Stripping Function
def skull_stripping(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    brain = cv2.bitwise_and(image, image, mask=mask)
    return brain

# Median Filtering Function
def apply_median_filter(image):
    return cv2.medianBlur(image, 5)

# Preprocess Images
def preprocess_images(img_dir, target_size=(224, 224)):
    images, labels = [], []
    class_names = sorted(os.listdir(img_dir))
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(img_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, target_size)
                img = skull_stripping(img)
                img = apply_median_filter(img)
                img = img_to_array(img)
                img = preprocess_input(img)
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Warning: {e}. Skipping {img_path}.")
    if len(images) == 0:
        raise ValueError("No valid images found in the dataset.")
    return np.array(images), np.array(labels), class_names

# Balance Dataset
def balance_data(X, y):
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, y_resampled = smote_tomek.fit_resample(X.reshape(X.shape[0], -1), y)
    X_resampled = X_resampled.reshape(-1, 224, 224, 3)
    return X_resampled, y_resampled

# Fine-tune DenseNet121
def fine_tune_densenet(num_classes):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers[-33:]:
        layer.trainable = True
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation('mish')(x)
    x = Dropout(0.3)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output_layer)
    return model

# Hyperparameter tuning for SVM model
def tune_svm(X_train, y_train):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear'], 'gamma': ['scale', 'auto']}
    svm = SVC(probability=True)
    grid_search = GridSearchCV(svm, param_grid, cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

if __name__ == "__main__":
    try:
        images, labels, class_names = preprocess_images('dataset/2DSagittal')
        images, labels = balance_data(images, labels)
        X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, stratify=labels, random_state=42)

        model = fine_tune_densenet(num_classes=len(class_names))
        model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, to_categorical(y_train), epochs=15, batch_size=32, validation_data=(X_test, to_categorical(y_test)))

        feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
        X_train_feats = feature_extractor.predict(X_train)
        X_test_feats = feature_extractor.predict(X_test)

        best_svm = tune_svm(X_train_feats, y_train)
        y_pred = best_svm.predict(X_test_feats)

        print("\nSVM Model Results:")
        print("Test Accuracy:", accuracy_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))
        print(classification_report(y_test, y_pred, target_names=class_names))

    except Exception as e:
        print(f"Error: {e}")


Error: [Errno 2] No such file or directory: 'dataset/2DSagittal'


In [7]:
!pip install imbalanced-learn scikit-learn keras matplotlib seaborn tensorflow

In [8]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report, roc_auc_score,
                             f1_score, roc_curve)
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from keras.applications import DenseNet121
from keras.applications.densenet import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Activation, Conv2D, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
from imblearn.combine import SMOTETomek
from sklearn.svm import SVC

# Skull Stripping Function
def skull_stripping(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    return cv2.bitwise_and(image, image, mask=mask)

# Median Filtering Function
def apply_median_filter(image):
    return cv2.medianBlur(image, 5)

# Preprocess Images
def preprocess_images(img_dir, target_size=(224, 224), do_preprocessing=True):
    images, labels = [], []
    class_names = sorted(os.listdir(img_dir))
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(img_dir, class_name)
        if not os.path.isdir(class_path): continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, target_size)
                if do_preprocessing:
                    img = skull_stripping(img)
                    img = apply_median_filter(img)
                img = img_to_array(img)
                img = preprocess_input(img)
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Warning: {e} -> Skipping {img_path}")
    return np.array(images), np.array(labels), class_names

# Balance Dataset
def balance_data(X, y):
    smote_tomek = SMOTETomek(random_state=42)
    X_res, y_res = smote_tomek.fit_resample(X.reshape(X.shape[0], -1), y)
    return X_res.reshape(-1, 224, 224, 3), y_res

# DenseNet121 Model
def fine_tune_densenet(num_classes):
    base = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))
    for layer in base.layers[-33:]:
        layer.trainable = True
    x = GlobalAveragePooling2D()(base.output)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base.input, outputs=out)

# Simple CNN baseline
def simple_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model

# SVM Hyperparameter Tuning
def tune_svm(X_train, y_train):
    param_grid = {'C':[0.1, 1, 10], 'kernel':['rbf'], 'gamma':['scale', 'auto']}
    grid = GridSearchCV(SVC(probability=True), param_grid, cv=3, n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid.best_estimator_

# Evaluation & Visuals
def evaluate_model(y_true, y_pred, y_prob, class_names, title="Confusion Matrix"):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("F1-Score:", f1_score(y_true, y_pred, average='weighted'))
    print("ROC-AUC:", roc_auc_score(y_true, y_prob, multi_class='ovr'))
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    sensitivity = np.diag(cm) / np.sum(cm, axis=1)
    specificity = [np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1)) / np.sum(np.delete(cm, i, axis=0)) for i in range(len(class_names))]

    for i, cls in enumerate(class_names):
        print(f"{cls} - Sensitivity: {sensitivity[i]:.2f}, Specificity: {specificity[i]:.2f}")

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

# Plot training history
def plot_history(history):
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title("Training History")
    plt.legend()
    plt.show()

if __name__ == "__main__":
    try:
        # Preprocessing & Dataset
        images, labels, class_names = preprocess_images("dataset/2DSagittal")
        images, labels = balance_data(images, labels)
        X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, stratify=labels, random_state=42)

        # DNN Model
        model = fine_tune_densenet(num_classes=len(class_names))
        model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
        history = model.fit(X_train, to_categorical(y_train), validation_data=(X_test, to_categorical(y_test)), epochs=10, batch_size=32)

        # Plot training performance
        plot_history(history)

        # SVM on extracted features
        extractor = Model(inputs=model.input, outputs=model.layers[-2].output)
        X_train_feat = extractor.predict(X_train)
        X_test_feat = extractor.predict(X_test)

        best_svm = tune_svm(X_train_feat, y_train)
        y_pred_svm = best_svm.predict(X_test_feat)
        y_prob_svm = best_svm.predict_proba(X_test_feat)

        print("\nSVM Evaluation:")
        evaluate_model(y_test, y_pred_svm, y_prob_svm, class_names)

        # Feature Importance (Permutation)
        print("\nTop 10 Important Features (SVM):")
        result = permutation_importance(best_svm, X_test_feat, y_test, n_repeats=10, random_state=42, n_jobs=-1)
        top_features = np.argsort(result.importances_mean)[::-1][:10]
        print("Feature indices:", top_features)

        # Baseline: Logistic Regression
        print("\nBaseline: Logistic Regression")
        X_flat = X_train_feat
        clf_lr = LogisticRegression(max_iter=1000)
        clf_lr.fit(X_flat, y_train)
        y_pred_lr = clf_lr.predict(X_test_feat)
        y_prob_lr = clf_lr.predict_proba(X_test_feat)
        evaluate_model(y_test, y_pred_lr, y_prob_lr, class_names, title="Logistic Regression")

        # Baseline CNN
        print("\nBaseline CNN:")
        cnn = simple_cnn((224,224,3), len(class_names))
        cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        cnn.fit(X_train, to_categorical(y_train), epochs=5, batch_size=32)
        y_pred_cnn = np.argmax(cnn.predict(X_test), axis=1)
        y_prob_cnn = cnn.predict(X_test)
        evaluate_model(y_test, y_pred_cnn, y_prob_cnn, class_names, title="Simple CNN")

    except Exception as e:
        print(f"Error: {e}")


Error: [Errno 2] No such file or directory: 'dataset/2DSagittal'


In [9]:
# Save the trained model
model.save('densenet_finetuned_model.h5')
print("Model saved successfully as 'densenet_finetuned_model.h5'")

NameError: name 'model' is not defined

In [ ]:
import matplotlib.pyplot as plt

# After model.fit(...)
history = model.fit(
    X_train, to_categorical(y_train),
    epochs=10,
    batch_size=32,
    validation_data=(X_test, to_categorical(y_test))
)

# Plot accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)  # Set y-axis range
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim(0, 0.5)  # Set y-axis range
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("training_plots.png")
plt.show()


In [ ]:
import joblib
joblib.dump(best_svm, "svm_model.pkl")
print("SVM model saved as 'svm_model.pkl'")

In [ ]:
!pip install --upgrade joblib scikit-learn

In [ ]:
from tensorflow.keras.models import load_model
import joblib # use joblib to load, not pickle

# Load fine-tuned DenseNet model
densenet_model = load_model('densenet_finetuned_model.h5')

# Load trained SVM model
# Use joblib.load to load the model since it was saved using joblib.dump
with open('svm_model.pkl', 'rb') as f:
    svm_model = joblib.load(f)

In [ ]:
import cv2
import numpy as np

def skull_stripping(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Otsu's thresholding
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Apply morphological operations to remove noise and fill holes
    kernel = np.ones((5, 5), np.uint8)
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)

    # Find contours and keep the largest one (assumed to be brain region)
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(gray)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

    # Mask the original image
    brain_only = cv2.bitwise_and(image, image, mask=mask)

    return brain_only


In [ ]:
def full_preprocess_pipeline(image_path):
    # Load image
    orig = cv2.imread(image_path)
    orig_resized = cv2.resize(orig, (224, 224))

    # Step 1: Skull stripping
    stripped = skull_stripping(orig_resized)

    # Step 2: Median filtering
    filtered = apply_median_filter(stripped)

    # Step 3: To array and preprocessing for DenseNet
    img_array = img_to_array(filtered)
    img_preprocessed = preprocess_input(img_array)

    return orig_resized, stripped, filtered, img_preprocessed


In [ ]:
def extract_features_and_visualize(image, model, svm, class_names):
    pred_probs = model.predict(np.expand_dims(image, axis=0))
    pred_class = np.argmax(pred_probs)

    # Grad-CAM
    cam = generate_grad_cam(model, image, pred_class)
    heatmap_img = overlay_heatmap((image + 1) * 127.5, cam)  # Assuming preprocess_input subtracts mean

    # Feature extraction
    feature_extractor = tf.keras.models.Model(inputs=model.input, outputs=model.layers[-2].output)
    features = feature_extractor.predict(np.expand_dims(image, axis=0))

    # SVM prediction
    svm_pred = svm.predict(features)[0]

    return heatmap_img, pred_class, svm_pred


In [ ]:
def demo_image_pipeline(image_path, model, svm, class_names):
    orig, stripped, filtered, preprocessed = full_preprocess_pipeline(image_path)
    heatmap_img, cnn_pred, svm_pred = extract_features_and_visualize(preprocessed, model, svm, class_names)

    # Plot all stages
    titles = ["Original", "Skull Stripped", "Median Filtered", "Grad-CAM"]
    images = [orig, stripped, filtered, heatmap_img]

    plt.figure(figsize=(12, 6))
    for i in range(4):
        plt.subplot(1, 4, i+1)
        plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
        plt.title(titles[i])
        plt.axis('off')
    plt.suptitle(f"CNN Predicted: {class_names[cnn_pred]}, SVM Predicted: {class_names[svm_pred]}")
    plt.tight_layout()
    plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import cv2

def generate_grad_cam(model, image, class_idx, layer_name=None):
    if layer_name is None:
        # Try to find the last convolutional layer automatically
        for layer in reversed(model.layers):
            if 'conv' in layer.name:
                layer_name = layer.name
                break

    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(tf.expand_dims(image, axis=0))
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)

    # Global average pooling on the gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]  # Remove batch dimension
    cam = np.zeros(conv_outputs.shape[:2], dtype=np.float32)

    for i, w in enumerate(pooled_grads):
        cam += w * conv_outputs[:, :, i]

    cam = np.maximum(cam, 0)  # ReLU
    cam = cam / (np.max(cam) + 1e-8)  # Normalize
    cam = cv2.resize(cam, (224, 224))  # Resize to image size # Remove .numpy() call as cam is already a NumPy array
    return cam

In [ ]:
demo_image_pipeline("image_0.jpg", densenet_model, svm_model, class_names)
demo_image_pipeline("image_0.jpg", densenet_model, svm_model, class_names)
demo_image_pipeline("image_0.jpg", densenet_model, svm_model, class_names)

In [ ]:
!git clone https://github.com/tejaswini16256/Brain-Tumor-Detection.git
%cd Brain-Tumor-Detection


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!ls "/content/drive/MyDrive"


 084RG7JD
'10 th marksheet.pdf'
'12 th marksheet.pdf'
 1A2A5361.JPG
'2025_Tejaswini_resume (1) (1) (10).pdf'
'2025_Tejaswini_resume (1) (1) (11).pdf'
'2025_Tejaswini_resume (1) (1) (12).pdf'
'2025_Tejaswini_resume (1) (1) (13).pdf'
'2025_Tejaswini_resume (1) (1) (14).pdf'
'2025_Tejaswini_resume (1) (1) (15).pdf'
'2025_Tejaswini_resume (1) (1) (16).pdf'
'2025_Tejaswini_resume (1) (1) (17).pdf'
'2025_Tejaswini_resume (1) (1) (1).pdf'
'2025_Tejaswini_resume (1)-1 (1).pdf'
'2025_Tejaswini_resume (1) (1) (2).pdf'
'2025_Tejaswini_resume (1) (1) (3).pdf'
'2025_Tejaswini_resume (1) (1) (4).pdf'
'2025_Tejaswini_resume (1) (1) (5).pdf'
'2025_Tejaswini_resume (1) (1) (6).pdf'
'2025_Tejaswini_resume (1) (1) (7).pdf'
'2025_Tejaswini_resume (1) (1) (8).pdf'
'2025_Tejaswini_resume (1) (1) (9).pdf'
'2025_Tejaswini_resume (1)-1.pdf'
'2025_Tejaswini_resume (1) (1).pdf'
'2025_Tejaswini_resume (1) (2).pdf'
'2025_Tejaswini_resume (1) (3).pdf'
'2025_Tejaswini_resume (1).pdf'
'21610013-assinment finale.docx'

In [17]:
!cp -r "/content/drive/MyDrive/Colab Notebooks/*" "/content/Brain-Tumor-Detection/"


cp: cannot stat '/content/drive/MyDrive/Colab Notebooks/*': No such file or directory
